# FIT5196 Assessment 2 Task2
#### Student Name:  Maryam Mahmoodi
#### Student ID:         30155843


Date: 13/06/2020

Version: 1.0

Environment: Python 3.8.5 and Jupyter notebook

Libraries used: please include the main libraries you used in your assignment here, e.g.,:
* pandas 
* re 
* numpy
* mpl
* plt
* pylab
* math
* statsmodels.api
* sklearn.linear_model
* datetime

In [1]:
from platform import python_version

print(python_version())

3.8.5


## Import libraries 

In [2]:
# Code to import libraries as you need in this assessment, e.g.,
import pandas as pd

# REGEX library to match strings
import re

# Mainly mathematical libraries to deal with stanstiscal modeling as well as maths
import numpy as np
import math
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

# datetime lib to deal with open and close date data
from datetime import datetime

# Sequence matcher is used to cleanse categorical data and tries to find duplicate rows which are very similar
# but distinguished mainly due to human error (e.g. typo)
from difflib import SequenceMatcher


# Better design plottings
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab
%matplotlib inline
%matplotlib inline
mpl.style.use( 'ggplot' )
#Notebook displace setting
from IPython.core.display import HTML
css = open('style/style-table.css').read() + open('style/style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

## I. Examin two data sets

In [3]:
# read in two data set
df1 = pd.read_csv('dataset1_solution.csv')
df2 = pd.read_csv("dataset2.csv")
# Display two dataframes together to compare
print('S1:' + str(df1.shape))
display(df1.head(3))
print('S2:' + str(df2.shape))
display(df2.head(3))

S1:(40162, 11)


,Id,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate,Source
0,12612628,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,25000.0,"12PM, 8th July 2013","12PM, 6th September 2013",CVLIBRARY.CO.UK
1,12612830,STRESS ENGINEER GLASGOW,GLASGOW,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,30000.0,"12AM, 30th January 2012","12AM, 30th March 2012",CVLIBRARY.CO.UK
2,12612844,MODELLING & SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,30000.0,"3PM, 21st December 2012","3PM, 20th January 2013",CVLIBRARY.CO.UK


S2:(334, 9)


,Opening,Closing,Job Title,Organisation,Location,Category,Salary per month,Fraction,Contract Type
0,2013-10-06 00:00:00,2013-12-05 00:00:00,Aviation loans administration,cer Financial,London,Finance and Accounting,2800,NaN,Contract
1,2012-10-03 12:00:00,2012-11-02 12:00:00,"Payroll Analyst City upto **** , ****",LMA Recruitment Ltd,London,Finance and Accounting,2917,NaN,Permanent
2,2012-01-01 00:00:00,2012-01-31 00:00:00,Investment Team Assistant for leading Private ...,Austin Andrew Ltd,London,Finance and Accounting,3750,NaN,Permanent


In [4]:
print(df1.info())
print(df2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40162 entries, 0 to 40161
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Id            40162 non-null  int64  
 1   Title         40161 non-null  object 
 2   Location      40162 non-null  object 
 3   Company       39176 non-null  object 
 4   ContractType  40162 non-null  object 
 5   ContractTime  40162 non-null  object 
 6   Category      40162 non-null  object 
 7   Salary        40162 non-null  float64
 8   OpenDate      40162 non-null  object 
 9   CloseDate     40162 non-null  object 
 10  Source        40162 non-null  object 
dtypes: float64(1), int64(1), object(9)
memory usage: 3.4+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Opening           334 non-null    object
 1   Closing           

## II. Resolve schema conflict

### Schema conflict 1: datetime format differs in S1 and S2

- Create consistency in Opening and Closing dates datetime format

In [5]:
# Change the column name in S2 to conform with matching column in S1
df2.rename(columns = {'Opening':'OpenDate'}, inplace = True)

In [6]:
# Change the column name in S2 to conform with matching column in S1
df2.rename(columns = {'Closing':'CloseDate'}, inplace = True)

In [7]:
# Create a function to manage day suffix for 1st, 2nd, and 3rd and rest (th) day of each month

def suffix(d):
    return 'th' if 11<=d<=13 else {1:'st',2:'nd',3:'rd'}.get(d%10, 'th')

def custom_strftime(format, t):
    return t.strftime(format).replace('{S}', str(t.day) + suffix(t.day))

In [8]:
# Following lines will match the date time based on the assessment's requirements

formatted_OpenDate=[]
for i in df2["OpenDate"]:
    try:
        new_open_date = custom_strftime('%-I%p, {S} %B %Y', datetime.strptime(i, '%Y-%m-%d %H:%M:%S'))
    except ValueError:
        new_open_date = 0
    formatted_OpenDate.append(new_open_date)
formatted_OpenDate = formatted_OpenDate
openDateS = pd.Series(data=formatted_OpenDate, index = df2.index)
df2['OpenDate'] = openDateS

In [9]:
# Following lines will match the date time based on the assessment's requirements

formatted_CloseDate=[]
for i in df2["CloseDate"]:
    try:
        new_close_date = custom_strftime('%-I%p, {S} %B %Y', datetime.strptime(i, '%Y-%m-%d %H:%M:%S'))
    except ValueError:
        new_close_date = 0
    formatted_CloseDate.append(new_close_date)
formatted_CloseDate = formatted_CloseDate
closeDateS = pd.Series(data=formatted_CloseDate, index = df2.index)
df2['CloseDate'] = closeDateS

### Schema conflict 2: Job title column has a different heading

- Change the Job Title to 'Title' in S2

In [10]:
df2.rename(columns = {'Job Title':'Title'}, inplace = True)

### Schema conflict 3: Company column has a different heading

- Change the name in S2

In [11]:
# Change the column name in S2 to conform with matching column in S1

df2.rename(columns = {'Organisation':'Company'}, inplace = True)

### Schema conflict 4: No conflict in Category schema



### Schema conflict 5: Salary
- Covert the salary per month in S2 to annual salary, and rename to be consistent

In [12]:
# covert the salary per month in S2 to annual salary, and rename to be consistent
df2['Salary per month'] = df2['Salary per month'].apply(lambda x: x*12)

df2.rename(columns = {'Salary per month':'Salary'}, inplace = True)

df2['Salary'].head()

0    33600
1    35004
2    45000
3    39996
4    39360
Name: Salary, dtype: int64

### Schema conflict 6: Contract type and time
    - Rename heading in S2

In [13]:
df2.rename(columns = {'Contract Type':'ContractTime'}, inplace = True)

In [14]:
df2.rename(columns = {'Fraction':'ContractType'}, inplace = True)

### Semantic mapping
- 'Id' and 'Source' columns need to be removed from S1 since they don't exist in S2
- The order of S2 needs to be aligned with S1

In [15]:
del df1['Id']

In [16]:
del df1['Source']

In [17]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334 entries, 0 to 333
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   OpenDate      334 non-null    object
 1   CloseDate     334 non-null    object
 2   Title         334 non-null    object
 3   Company       334 non-null    object
 4   Location      334 non-null    object
 5   Category      334 non-null    object
 6   Salary        334 non-null    int64 
 7   ContractType  3 non-null      object
 8   ContractTime  327 non-null    object
dtypes: int64(1), object(8)
memory usage: 23.6+ KB


In [18]:
df2 = df2[['Title',
'Location',
'Company',
'ContractType',
'ContractTime',
'Category',
'OpenDate',
'CloseDate',
'Salary']]

In [19]:
df1.head(2)

,Title,Location,Company,ContractType,ContractTime,Category,Salary,OpenDate,CloseDate
0,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,25000.0,"12PM, 8th July 2013","12PM, 6th September 2013"
1,STRESS ENGINEER GLASGOW,GLASGOW,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,30000.0,"12AM, 30th January 2012","12AM, 30th March 2012"


In [20]:
df2.head(2)

,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
0,Aviation loans administration,London,cer Financial,NaN,Contract,Finance and Accounting,"12AM, 6th October 2013","12AM, 5th December 2013",33600
1,"Payroll Analyst City upto **** , ****",London,LMA Recruitment Ltd,NaN,Permanent,Finance and Accounting,"12PM, 3rd October 2012","12PM, 2nd November 2012",35004


In [21]:
print(df1.shape)
print(df2.shape)

(40162, 9)
(334, 9)


## Task 2. Integration
- First, I need to audit if data in each column have a similar format and if not, normalise them
- Second, we merge the to data sets
- and at the end, we remove duplicates

In [22]:
# now adapt the datatypes in S2 to match S1
for y in df1.columns:
    if df2[y].dtype != df1[y].dtype:
        print( "Column " + y + " in df2: "+ str(df2[y].dtype) + " to " + str(df1[y].dtype))
        df2[y] = df2[y].astype(df1[y].dtype) 

Column Salary in df2: int64 to float64


### Merging the 2 data sets
- Due to the fact that the column names are identical in both data sets and their records are cleansed and similar in terms of formatting, I will use the 'concat' function to attach S2 at the bottom of S1

In [23]:
df = pd.concat([df1,df2],sort=True)
print(df.shape)
df.head()

(40496, 9)


,Category,CloseDate,Company,ContractTime,ContractType,Location,OpenDate,Salary,Title
0,Engineering Jobs,"12PM, 6th September 2013",GREGORY MARTIN INTERNATIONAL,PERMANENT,NON-SPECIFIED,DORKING,"12PM, 8th July 2013",25000.0,ENGINEERING SYSTEMS ANALYST
1,Engineering Jobs,"12AM, 30th March 2012",GREGORY MARTIN INTERNATIONAL,PERMANENT,NON-SPECIFIED,GLASGOW,"12AM, 30th January 2012",30000.0,STRESS ENGINEER GLASGOW
2,Engineering Jobs,"3PM, 20th January 2013",GREGORY MARTIN INTERNATIONAL,PERMANENT,NON-SPECIFIED,HAMPSHIRE,"3PM, 21st December 2012",30000.0,MODELLING & SIMULATION ANALYST
3,Engineering Jobs,"3PM, 6th February 2014",GREGORY MARTIN INTERNATIONAL,PERMANENT,NON-SPECIFIED,SURREY,"3PM, 8th December 2013",27500.0,ENGINEERING SYSTEMS ANALYST MATHEMATICAL MODELLER
4,Engineering Jobs,"12PM, 1st May 2013",GREGORY MARTIN INTERNATIONAL,PERMANENT,NON-SPECIFIED,SURREY,"12PM, 2nd March 2013",25000.0,PIONEER MISER ENGINEERING SYSTEMS ANALYST


In [24]:
df = df[['Title',
'Location',
'Company',
'ContractType',
'ContractTime',
'Category',
'OpenDate',
'CloseDate',
'Salary']]
df.head()

,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
0,ENGINEERING SYSTEMS ANALYST,DORKING,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,"12PM, 8th July 2013","12PM, 6th September 2013",25000.0
1,STRESS ENGINEER GLASGOW,GLASGOW,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,"12AM, 30th January 2012","12AM, 30th March 2012",30000.0
2,MODELLING & SIMULATION ANALYST,HAMPSHIRE,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,"3PM, 21st December 2012","3PM, 20th January 2013",30000.0
3,ENGINEERING SYSTEMS ANALYST MATHEMATICAL MODELLER,SURREY,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,"3PM, 8th December 2013","3PM, 6th February 2014",27500.0
4,PIONEER MISER ENGINEERING SYSTEMS ANALYST,SURREY,GREGORY MARTIN INTERNATIONAL,NON-SPECIFIED,PERMANENT,Engineering Jobs,"12PM, 2nd March 2013","12PM, 1st May 2013",25000.0


### Resolve data conflicts:
- At this stage, I have a finalised data frame. 
- However, I need to audit and remove any type of duplication in the finalised data set.
- I will take two approach to ensure duplication removal:
     1. I use all columns to look for duplicates, and then,
     2. Based on each column's attribute, I use common sense to check if any 2 columns can be used as ordered
        pair to find duplicates and if yes, remove the duplicate

In [25]:
# check duplication
cols = df.columns.difference([]) # use all columns to check duplicates
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 6 duplicate records found


,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
39645,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,HACKNEY,BARCLAYS,NON-SPECIFIED,NON-SPECIFIED,IT Jobs,"3PM, 23rd December 2012","3PM, 21st February 2013",36000.0
40161,BARCLAYS FUTURE LEADERS DEVELOPMENT PROGRAMMES,HACKNEY,BARCLAYS,NON-SPECIFIED,NON-SPECIFIED,IT Jobs,"3PM, 23rd December 2012","3PM, 21st February 2013",36000.0
34408,PENSIONS ADMINISTRATORS TEMPORARYCONTRACT,UK,ABENEFIT2U,NON-SPECIFIED,CONTRACT,Accounting & Finance Jobs,"3PM, 1st August 2013","3PM, 31st August 2013",24000.0
40159,PENSIONS ADMINISTRATORS TEMPORARYCONTRACT,UK,ABENEFIT2U,NON-SPECIFIED,CONTRACT,Accounting & Finance Jobs,"3PM, 1st August 2013","3PM, 31st August 2013",24000.0
25098,SENIOR FINANCIAL ADVISOR,LONDON,FRAM EXECUTIVE SEARCH,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"12AM, 26th January 2013","12AM, 25th February 2013",40000.0
40160,SENIOR FINANCIAL ADVISOR,LONDON,FRAM EXECUTIVE SEARCH,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"12AM, 26th January 2013","12AM, 25th February 2013",40000.0


In [26]:
df = df.drop_duplicates(cols, keep='last')
df.shape

(40493, 9)

In [27]:
# check duplication - whether the same title is advertised multiple times in the same date.
cols = ['Title','OpenDate']  
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 4 duplicate records found


,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
20864,MANDARIN SPEAKING INTERNAL AUDITOR,LONDON,PEOPLE FIRST RECRUITMENT LTD,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"12PM, 25th July 2013","12PM, 23rd September 2013",22500.0
145,MANDARIN SPEAKING INTERNAL AUDITOR,London,People First Recruitment Limited,NaN,Permanent,Finance and Accounting,"12PM, 25th July 2013","12PM, 23rd September 2013",22500.0
11072,MOTOR CLAIMS COMPLAINTS TEAM LEADER,UK,CLARK JAMES LTD,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"12AM, 2nd February 2013","12AM, 3rd April 2013",26500.0
295,MOTOR CLAIMS COMPLAINTS TEAM LEADER,UK,Clark James Ltd,NaN,Permanent,Finance and Accounting,"12AM, 2nd February 2013","12AM, 3rd April 2013",26496.0


- Interestingly, we find two duplicate pairs, therefore, I remove one from each and keep the last one.

In [28]:
df = df.drop_duplicates(cols, keep='last')
df.shape

(40491, 9)

In [29]:
# check duplication - Similar to previous line, checking the title and Closing date pair
cols = ['Title','CloseDate'] 
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(10)

There are 0 duplicate records found


,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary


In [30]:
# check duplication - Similar to previous line, checking the title and Company date pair
cols = ['Title', 'Company'] 
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(12)

There are 4 duplicate records found


,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
13485,BUSINESS SUPPORT ANALYST WITH SQL & CRYSTAL RE...,NORTH LAMBETH,THE OAKLAND PARTNERSHIP,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"3PM, 1st January 2012","3PM, 1st March 2012",45000.0
2214,BUSINESS SUPPORT ANALYST WITH SQL & CRYSTAL RE...,NORTH LONDON,THE OAKLAND PARTNERSHIP,NON-SPECIFIED,PERMANENT,Accounting & Finance Jobs,"12AM, 26th January 2012","12AM, 25th February 2012",45000.0
22266,DATA CENTRE ENGINEER ELECTRICAL & MECHANICAL,UK,OMNI RESOURCE MANAGEMENT SOLUTIONS,NON-SPECIFIED,PERMANENT,Engineering Jobs,"3PM, 22nd August 2012","3PM, 20th November 2012",32000.0
32453,DATA CENTRE ENGINEER ELECTRICAL & MECHANICAL,YORKSHIRE & HUMBERSIDE,OMNI RESOURCE MANAGEMENT SOLUTIONS,FULLTIME,PERMANENT,Engineering Jobs,"3PM, 22nd November 2013","3PM, 22nd December 2013",28500.0


- Four duplicates are identified, but we DO NOT remove them since they are not really duplicates.

In [31]:
# check duplication - Similar to previous line, checking the title and Location date pair
cols = ['Title', 'Location'] 
duplicates = df[df.duplicated(cols,keep=False)]
print( "There are "+ str(len(duplicates)) + " duplicate records found")
duplicates.sort_values(by=duplicates.columns.tolist()).head(12)

There are 12 duplicate records found


,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate,Salary
19031,FINANCE PLANNING & SALES COORDINATION ASSISTANT,SLOUGH,UKSTAFFSEARCH,FULLTIME,PERMANENT,Sales Jobs,"12PM, 9th January 2013","12PM, 10th March 2013",24000.0
9212,FINANCE PLANNING & SALES COORDINATION ASSISTANT,SLOUGH,NaN,NON-SPECIFIED,PERMANENT,IT Jobs,"12AM, 20th June 2012","12AM, 18th September 2012",23000.0
34762,GENERAL MANAGER BRANDED PUB & RESTAURANT LIVE ...,SOUTH EAST LONDON,DETAIL 2 RETAIL LTD,NON-SPECIFIED,PERMANENT,Hospitality & Catering Jobs,"12AM, 27th August 2012","12AM, 26th September 2012",24500.0
31111,GENERAL MANAGER BRANDED PUB & RESTAURANT LIVE ...,SOUTH EAST LONDON,DETAIL2LEISURE,NON-SPECIFIED,PERMANENT,Hospitality & Catering Jobs,"12AM, 30th October 2012","12AM, 29th December 2012",24500.0
36663,MARKETING PR & COMMUNICATIONS MANAGER,BLACKPOOL,SPRING PERSONNEL,NON-SPECIFIED,PERMANENT,"PR, Advertising & Marketing Jobs","12PM, 2nd February 2013","12PM, 4th March 2013",22500.0
19132,MARKETING PR & COMMUNICATIONS MANAGER,BLACKPOOL,NaN,FULLTIME,NON-SPECIFIED,"PR, Advertising & Marketing Jobs","3PM, 28th September 2013","3PM, 28th October 2013",22500.0
34721,PLUMBING & HEATING ASSESSOR,UK,ALPHA RECRUITMENT,NON-SPECIFIED,PERMANENT,Teaching Jobs,"3PM, 22nd March 2012","3PM, 21st May 2012",39936.0
24611,PLUMBING & HEATING ASSESSOR,UK,INTERACTION ROMFORD,NON-SPECIFIED,CONTRACT,Teaching Jobs,"3PM, 1st October 2013","3PM, 15th October 2013",39936.0
26000,QUALITY ENGINEER TEST & WARRANTY,MILTON KEYNES,PREMIERE PEOPLE DAVENTRY,NON-SPECIFIED,PERMANENT,Engineering Jobs,"3PM, 14th December 2013","3PM, 14th March 2014",27500.0
25548,QUALITY ENGINEER TEST & WARRANTY,MILTON KEYNES,NaN,FULLTIME,NON-SPECIFIED,Engineering Jobs,"3PM, 21st March 2013","3PM, 20th April 2013",27500.0


- This time 12 duplicates are identified by Python but none of them are really duplicate.
    Therefore, we consider this the most normalised data set and save it as final outcome.

## The Global Key

In [32]:
df.describe(include=['O'])

,Title,Location,Company,ContractType,ContractTime,Category,OpenDate,CloseDate
count,40490,40491,39505,40160,40484,40491,40491,40491
unique,40473,517,6728,4,5,11,2194,2409
top,VISUAL BASIC DATABASE DEVELOPER VBNET & SQL SE...,UK,NONSPECIFIED,NON-SPECIFIED,PERMANENT,IT Jobs,"3PM, 15th April 2012","12AM, 8th December 2013"
freq,2,6139,2717,30486,26400,11145,38,35


- Based on ordered pair matching in 'resolve data conflicts' section, it has become vivid that 'Title' will be the best candidate to be considered as globar primary key. Although 'ID' could act as a primary key due to its uniqueness, I have to select 'title' since 'ID' is not available and there's no better attribute to play that role. Moreover, in the table above, we see that the only column which has the closest number of unique and count is also the 'Title' column. Hence, my suggestion will be 'Title' as the global key.

## Saving data
Save the integrated data

In [33]:
df.to_csv("dataset_integrated.csv",index = False)

## Summary
in Task 2 of assessment 2, the focus was mainly on data and schema conflict management as well as efficient integration. Data wrangling in terms of cleansing and formatting was not required.
I started with importing the two data frames and set them as the two schema. Step by step, I compared their attributes and resolved their schema conflicts. To make them compatible, I removed two columns from schema 1 and merged them together using concat function. At the end, I resolved any data conflict by pair matching columns, and found and removed duplicate rows. The output can be found in the same directory of this code in the csv format named 'dataset_integrated'.